In [35]:
pip install rtree

Note: you may need to restart the kernel to use updated packages.


In [49]:
# Ensure that the 'rtree' library is installed. If not, run: pip install rtree
import trimesh
import numpy as np
import open3d as o3d
import plotly.graph_objects as go

# Load the OBJ model
obj_path = r'D:\IaaC\RESEARCH\GITHUB\Octopusie\sahils experiments\Reference files\3d model for detection.obj'  # Replace with your OBJ file path
mesh = trimesh.load(obj_path)

# Extract vertices and faces
vertices = mesh.vertices
faces = mesh.faces

# Edge Detection
edges = mesh.edges_unique
edge_vertices = vertices[edges]

# Get user input for floor height
num_floors_input = int(input('Enter the desired number of floors: '))

# User-defined Floor Division with Single Floor Height
def divide_into_floors(vertices, num_floors):
    # Find the bounding box of the model
    min_z = np.min(vertices[:, 2])
    max_z = np.max(vertices[:, 2])

    # Calculate the floor height
    floor_height = (max_z - min_z) / num_floors

    # Create floor levels
    floor_levels = [min_z + i * floor_height for i in range(num_floors + 1)]

    return floor_levels

# Divide into floors
floor_levels = divide_into_floors(vertices, num_floors_input)
print('Floor Levels:', floor_levels)

# Identify Floors and Walls (more sophisticated approach)
def is_floor(face_normal, normal_threshold=0.1):
    # Check if the face normal is mostly pointing up or down
    return abs(face_normal[2]) > (1 - normal_threshold)

def is_wall(face_normal, normal_threshold=0.1):
    # Check if the face normal is mostly horizontal
    return abs(face_normal[2]) < normal_threshold

floor_faces = set()
wall_faces = set()

# Adjacency
mesh.process() # Compute the face adjacency before accessing it
face_adjacency = mesh.face_adjacency

def group_connected_faces(faces, normals, similarity_threshold=0.95):
    groups = []
    visited = set()

    for face_index in faces:
        if face_index in visited:
            continue

        group = {face_index}
        queue = [face_index]
        visited.add(face_index)

        while queue:
            current_face = queue.pop(0)
            # Check if current_face is within the bounds of face_adjacency
            if current_face >= len(face_adjacency):
                continue
            neighbors = mesh.face_adjacency[current_face]

            if neighbors is None:
                continue

            for neighbor in neighbors:
                if neighbor in visited or neighbor in group:
                    continue

                if np.dot(normals[current_face], normals[neighbor]) > similarity_threshold:
                    group.add(neighbor)
                    queue.append(neighbor)
                    visited.add(neighbor)

        groups.append(list(group))

    return groups

all_faces = set(range(len(faces)))
floor_normals = [mesh.face_normals[i] for i in all_faces if is_floor(mesh.face_normals[i])]
floor_faces_indices = [i for i in all_faces if is_floor(mesh.face_normals[i])]
wall_normals = [mesh.face_normals[i] for i in all_faces if is_wall(mesh.face_normals[i])]
wall_faces_indices = [i for i in all_faces if is_wall(mesh.face_normals[i])]

floor_groups = group_connected_faces(floor_faces_indices, mesh.face_normals)
wall_groups = group_connected_faces(wall_faces_indices, mesh.face_normals)

num_floors = len(floor_groups)
num_walls = len(wall_groups)

# Print the number of walls and floors
print(f'Number of floor faces: {num_floors}')
print(f'Number of wall faces: {num_walls}')

# Visualization using Plotly
def visualize_floors(vertices, faces, floor_levels):
    # Create a Plotly figure
    fig = go.Figure()

    # Add the base mesh
    fig.add_trace(go.Mesh3d(x=vertices[:, 0], y=vertices[:, 1], z=vertices[:, 2],
                            i=faces[:, 0], j=faces[:, 1], k=faces[:, 2],
                            opacity=0.1, color='lightgrey'))

    # Find the bounds of the model
    x_min, x_max = np.min(vertices[:, 0]), np.max(vertices[:, 0])
    y_min, y_max = np.min(vertices[:, 1]), np.max(vertices[:, 1])

    # Create floor surfaces
    for i in range(len(floor_levels) - 1):
        z_level = floor_levels[i]

        # Create a grid of points for the floor surface
        x = np.linspace(x_min, x_max, 50)
        y = np.linspace(y_min, y_max, 50)
        X, Y = np.meshgrid(x, y)
        Z = np.full_like(X, z_level)

        # Flatten the grid for point querying
        points = np.stack([X.flatten(), Y.flatten(), Z.flatten()], axis=-1)

        # Check which points are inside the mesh
        inside_points = mesh.contains(points)

        # Reshape the inside points to the grid shape
        inside_grid = inside_points.reshape(X.shape)

        # Mask the X, Y, and Z grids
        X_masked = np.where(inside_grid, X, np.nan)
        Y_masked = np.where(inside_grid, Y, np.nan)
        Z_masked = np.where(inside_grid, Z, np.nan)

        # Add the masked surface to the figure
        fig.add_trace(go.Surface(x=X_masked, y=Y_masked, z=Z_masked, opacity=0.5, colorscale=[[0, 'blue'], [1, 'blue']]))

    # Update layout for interactive rotation
    fig.update_layout(scene=dict(aspectmode='data'))

    fig.show()

# Visualize the floors
visualize_floors(vertices, faces, floor_levels)

# Visualization (Open3D) - Commented out
mesh_o3d = o3d.io.read_triangle_mesh(obj_path)
# o3d.visualization.draw_geometries([mesh_o3d]) 

Floor Levels: [np.float64(0.0), np.float64(10.0), np.float64(20.0), np.float64(30.0), np.float64(40.0), np.float64(50.0)]
Number of floor faces: 6
Number of wall faces: 7
